# Visualización de los datos de la epidemia COVID-19

In [1]:
import requests
import ipywidgets
import json
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import datetime

source = "https://pomber.github.io/covid19/timeseries.json"
r = requests.get(source)
data_covid = r.json()

country_list = [country for country in data_covid.keys()]
country_list.sort()

# data_covid  = {country1: [{'date': 'yyyy-m-dd', 'confirmed': ?, 'deaths': ?, 'recovered': ?}, ...], 
#                country2: [{'date': 'yyyy-m-dd', 'confirmed': ?, 'deaths': ?, 'recovered': ?}, ...], ...}

DropDown1 = ipywidgets.Dropdown(options=country_list, description="Paises")   # display(ListBox)
CheckBox1 = ipywidgets.Checkbox(value=True, description="Mostrar Confirmados")
CheckBox2 = ipywidgets.Checkbox(value=True, description="Mostrar Fallecidos")
CheckBox3 = ipywidgets.Checkbox(value=True, description="Mostrar Recuperados")
RadioButton1 = ipywidgets.RadioButtons(options=["Lineal", "Logaritmica"], description="Escala Y")

GUI = ipywidgets.HBox([ipywidgets.VBox([DropDown1, RadioButton1]), 
                       ipywidgets.VBox([CheckBox1, CheckBox2, CheckBox3])])

#@ipywidgets.interact(country=DropDown1, 
#                     show_confirmed=CheckBox1,
#                     show_deaths=CheckBox2,
#                     show_recovered=CheckBox3, 
#                     yaxis_type=RadioButton1)
def plot_covid_data(country, show_confirmed, show_deaths, show_recovered, yaxis_type):
    country_data_list = data_covid.get(country)
    date_data = [data_dict.get('date') for data_dict in country_data_list]
    confirmed_data = [data_dict.get('confirmed') for data_dict in country_data_list]
    death_data = [data_dict.get('deaths') for data_dict in country_data_list]
    recovered_data = [data_dict.get('recovered') for data_dict in country_data_list]
    n_elements = len(country_data_list)
    
    text_data = f"Fecha: {date_data[-1]}:\n * Confirmados: {confirmed_data[-1]:,}\n * Fallecidos: {death_data[-1]:,}\n * Recuperados: {recovered_data[-1]:,}"
    
    data_array = np.concatenate([np.array(confirmed_data).reshape(n_elements, 1), 
                                 np.array(death_data).reshape(n_elements, 1), 
                                 np.array(recovered_data).reshape(n_elements, 1)], axis=1)
    
    dates = [datetime.datetime.strptime(d, "%Y-%m-%d").date() for d in date_data]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    if show_confirmed:
        if yaxis_type == 'Lineal':
            ax.plot(dates, data_array[:,0], '-o', label="Confirmados")
        else:
            ax.semilogy(dates, data_array[:,0], '-o', label="Confirmados")
    
    if show_deaths:
        if yaxis_type == 'Lineal':
            ax.plot(dates, data_array[:,1], '-s', label="Fallecidos")
        else:
            ax.semilogy(dates, data_array[:,1], '-s', label="Fallecidos")
                
    if show_recovered:
        if yaxis_type == 'Lineal':
            ax.plot(dates, data_array[:,2], '-^', label="Recuperados")
        else:
            ax.semilogy(dates, data_array[:,2], '-^', label="Recuperados")
    
    ax.set_title(f"Datos registrados por pais - {country}")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Poblacion")
    ax.legend()
    ax.grid(linestyle='--', which='both')
    
    ax.text(ax.get_xlim()[1] + 3, 3.5*ax.get_ylim()[1]/4, text_data, bbox={'facecolor': 'pink'})
    
    plt.show()
    

out = ipywidgets.interactive_output(plot_covid_data, {'country': DropDown1,
                                                      'show_confirmed': CheckBox1,
                                                      'show_deaths': CheckBox2,
                                                      'show_recovered': CheckBox3, 
                                                      'yaxis_type': RadioButton1})
display(GUI, out)

Output()

# Modelamiento matemático de epidemias (Modelo SIR o modelo simple de Kermack y McKendrick)

El modelo básico de epidemiología se conoce como Modelo SIR, donde las siglas indican la cantidad de Suceptibles, Infectados y Recuperados (donde se contabilizan aquellos que superaron la enfermedad tanto como los que fallecieron como resultado de esta).



<p>$\frac{dS}{dt} = -\frac{\beta IS}{N}$</p>
<p>$\frac{dI}{dt} = \frac{\beta IS}{N} - \gamma I$</p>
<p>$\space\space \frac{dR}{dt} = \gamma I$</p>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets
from IPython.display import display

N = 1           # Poblacion
#beta = 3.2     # Tasa de infeccion
#gamma = 0.23   # Tasa de Recuperacion

# def dU_dt(U, t):
#    Y = np.zeros(3)
#    Y[0] = -beta * U[0] * U[1]
#    Y[1] = beta * U[0] * U[1] - gamma * U[1]
#    Y[2] = gamma * U[1]
#    return Y

Beta = ipywidgets.FloatSlider(min=0.1, max=5.0, step=0.01, value=2.5, description="Transm")
Gamma = ipywidgets.FloatSlider(min=0.1, max=1.0, step=0.01, value=0.35, description="Recup")
Tmax = tmax=ipywidgets.IntSlider(min=1, max=100, value=50, description="Tiempo")
GUI = ipywidgets.HBox([Beta, Gamma, Tmax])

def plot_SIR(beta, gamma, tmax):
    
    def dU_dt(U, t):
        Y = np.zeros(3)
        Y[0] = -beta * U[0] * U[1]
        Y[1] = beta * U[0] * U[1] - gamma * U[1]
        Y[2] = gamma * U[1]
        return Y

    U0 = [0.99, 0.01, 0]   # Sini, Sin, Rini
    t = np.arange(0, tmax, 0.1)
    
    Us = odeint(dU_dt, U0, t)

    Ss = N * Us[:,0]
    Is = N * Us[:,1]
    Rs = N * Us[:,2]

    Imax = np.max(Is)
    r0 = beta / gamma   # Numero reproductivo basico (r = 2): r > 1 (Epidemia)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(t, Ss, label='Suceptibles')
    ax.plot(t, Is, label='Infectados')
    ax.plot(t, Rs, label='Recuperados')
    ax.legend()
    ax.set_xlabel("Tiempo")
    ax.set_ylabel("Poblacion")
    ax.grid()
    plt.show()
    
    print(f"r0 = {r0:.4f}, Max Infectados = {round(Imax):,.0f}")
    
    
out = ipywidgets.interactive_output(plot_SIR, {'beta': Beta, 'gamma': Gamma, 'tmax': Tmax})
display(GUI, out)

Output()